In [1]:
        import numpy as np
        import matplotlib.pyplot as plt
        from sklearn.metrics import roc_curve, auc,roc_auc_score
        from sklearn.linear_model import LinearRegression

In [2]:
import tensorflow.keras
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Dropout, ReLU, ELU, PReLU, Input, Concatenate, Lambda
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import mse, binary_crossentropy
import tensorflow as tf

In [3]:
def myloss2d(y_true, y_pred, alpha = 0.0):
    #alpha determines the amount of decorrelation; 0 means no decorrelation.
    
    #We want to learn f(g(x)) = x with g != identity and g(x) and x should have the same probability density.
    #g(x) = y_pred[:,0]
    #f(g(x)) = y_pred[:,1]
    #h(x) = y_pred[:,2]
    #h(g(x)) = y_pred[:,3]
    
    myoutput =  mse(y_true[:,0],y_pred[:,1]) \
                - alpha*binary_crossentropy(y_pred[:,2],K.ones_like(y_pred[:,2])) \
                - alpha*binary_crossentropy(y_pred[:,3],0.*K.ones_like(y_pred[:,2]))
    return myoutput

In [47]:
num_iters = 10
training = 20
zero = [0]*100000
slopes = []
intercepts = []

for j in range(num_iters):
    print("on j=",j)
    x0 = np.random.normal(0,1,100000)
    x1 = np.random.normal(0,2,100000)
    X = np.concatenate((np.c_[x0, zero], np.c_[zero, x1]))
    np.random.shuffle(X)
    
    myinput_2d = Input(shape=(2,))
    encoded_2d = Dense(5, activation='elu')(myinput_2d)
    encoded_2d = Dense(5, activation='elu')(encoded_2d)
    encoded_2d = Dense(2, activation='linear')(encoded_2d)

    encoder_2d = Model(myinput_2d, encoded_2d)
    encoder2_2d = encoder_2d(encoder_2d(myinput_2d))
    autoencoder_2d = Model(myinput_2d, encoder2_2d)

    combinedModel_2d = Model(myinput_2d,Concatenate(axis=-1)([encoded_2d, encoder2_2d]))

    myinput_classify_2d = Input(shape=(2,))
    myclassifier_2d = Dense(128, activation='elu')(myinput_classify_2d)
    myclassifier_2d = Dense(64, activation='elu')(myclassifier_2d)
    myclassifier_2d = Dense(1, activation='sigmoid')(myclassifier_2d)
    myclassifier_model_2d = Model(myinput_classify_2d, myclassifier_2d)
    myclassifier_input_2d = myclassifier_model_2d(myinput_2d)
    myclassifier_encoded_2d = myclassifier_model_2d(encoded_2d)

    combinedModel_classifier_2d = Model(myinput_2d,Concatenate(axis=-1)([encoded_2d, encoder2_2d, myclassifier_input_2d, myclassifier_encoded_2d]))

    preds = []
    preds += [encoder_2d.predict(X)]

    for i in range(training):
        #Now, train the classifier
        encoded_x_2d = encoder_2d.predict(X)
        myclassifier_model_2d.compile(optimizer='adam', loss='binary_crossentropy')
        myclassifier_model_2d.fit(np.concatenate([X,encoded_x_2d]),np.concatenate([np.ones(len(X)),np.zeros(len(X))]), epochs=1, batch_size=int(0.01*len(X)), verbose=0)

        #Now, update the autoencoder
        for layer in myclassifier_model_2d.layers[:]:
            layer.trainable = False

        combinedModel_classifier_2d.compile(optimizer='adam', loss=lambda y_true, y_pred: myloss2d(y_true, y_pred))
        combinedModel_classifier_2d.fit(X,X, epochs=1, batch_size=int(0.01*len(X)), verbose=0)

        autoencoder_2d.compile(optimizer='adam', loss='mse')
        autoencoder_2d.fit(X,X, epochs=5, batch_size=int(0.01*len(x0)), verbose=0)

        preds += [encoder_2d.predict(X)]

        for layer in myclassifier_model_2d.layers[:]:
            layer.trainable = True
            
            
            
    model = LinearRegression().fit(X, preds[-1]);
    slopes.append(model.coef_);
    intercepts.append(model.intercept_)
    

on j= 0


NameError: name 'around' is not defined

In [28]:
for i in range(num_iters):
    det = np.linalg.det(slopes[i])
    print(round(det, 0))

-1.0
1.0
1.0
-1.0
-1.0
1.0
1.0
-1.0
-1.0
1.0


In [42]:
print("Slopes:")
for i in range(num_iters):
    s = [[str(e) for e in row] for row in slopes[i]]
    lens = [max(map(len, col)) for col in zip(*s)]
    fmt = '\t'.join('{{:{}}}'.format(x) for x in lens)
    table = [fmt.format(*row) for row in s]
    print ("\n".join(table))
    print("\n")
    
print("Intercepts:")
for i in range(num_iters):
    s = [[str(e) for e in row] for row in intercepts[i]]
    lens = [max(map(len, col)) for col in zip(*s)]
    fmt = '\t'.join('{{:{}}}'.format(x) for x in lens)
    table = [fmt.format(*row) for row in s]
    print ("\n".join(table))
    print("\n")
    

slopes:
-0.09278478527234023	0.9843554811580026 
0.9992958749017833  	0.09981565741990134


0.996219824011967     	-7.252000201800778e-05
-0.0010184638323343194	0.9981393463719728    


0.9937786860825532    	0.0018701124955965381
-0.0018036101566574131	0.9959162301705138   


0.12135622184597479	0.7255222110337151  
1.3774880135779222 	-0.10862585202451683


0.6157793517012241	0.4740765417630849 
1.2395782399295983	-0.6028152808746761


0.9993351296475016   	0.001222516721229062
0.0022885791141539426	0.9988599081316957  


-0.988037076305449 	0.012396866506691495
0.11694926594647093	-0.9905298537731809 


0.06425309650154268	0.9525106710159424   
1.0270718908730325 	-0.027679601969283495


0.7667731650052971	0.35158197525753526
1.4697452952259606	-0.6577105911310936


0.9967595756908981    	0.0011562549183422365
-0.0037155190499997457	0.9968674134796016   


